In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
unibank = pd.read_csv('../input/bank-loan-classification/UniversalBank.csv')

In [ ]:
unibank.head()

In [ ]:
## Check the datatype of each variable
unibank.dtypes

In [ ]:
## Drop columns which are not significant
del unibank['ID']
del unibank["ZIP Code"]

In [ ]:
unibank.head()

In [ ]:
#Coorelation Plot
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt

df = pd.DataFrame(unibank,columns=['Age','Experience','Income','CCAvg','Mortgage'])

corrMatrix = df.corr()
sn.heatmap(corrMatrix, annot=True)
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split

X = unibank.copy().drop("Personal Loan",axis=1)
y = unibank["Personal Loan"]

## Split the data into trainx, testx, trainy, testy with test_size = 0.20 using sklearn
trainx, testx, trainy, testy = train_test_split(X, y, test_size=0.20)

## Print the shape of X_train, X_test, y_train, y_test
print(trainx.shape)
print(testx.shape)
print(trainy.shape)
print(testy.shape)

**Logistic Regression**

In [ ]:
from sklearn.preprocessing import StandardScaler 
sc_x = StandardScaler() 
trainx = sc_x.fit_transform(trainx)  
testx = sc_x.transform(testx)
from sklearn.linear_model import LogisticRegression 
classifier = LogisticRegression(random_state = 10) 
classifier.fit(trainx, trainy)

In [ ]:
y_pred = classifier.predict(testx)
from sklearn.metrics import accuracy_score 
print ("Accuracy : ", accuracy_score(testy, y_pred))

**Support Vector Machine (SVM)**

In [ ]:
from sklearn.svm import SVC

## Create an SVC object and print it to see the default arguments
svc = SVC()
print(svc)

In [ ]:
## Fit
svc.fit(trainx,trainy)

## Predict
train_predictions = svc.predict(trainx)
test_predictions = svc.predict(testx)

In [ ]:
model_C = SVC(C=2)
model_C.fit(trainx, trainy)
model_C.score(testx, testy)

In [ ]:
model_C = SVC(C=1)
model_C.fit(trainx, trainy)
model_C.score(testx, testy)

In [ ]:
model_g = SVC(gamma=0.002)
model_g.fit(trainx, trainy)
model_g.score(testx, testy)

In [ ]:
model_g = SVC(gamma=0.001)
model_g.fit(trainx, trainy)
model_g.score(testx, testy)

In [ ]:
#Most optimised model at C=2 and gamma= 0.002
#Kernel
model_linear_kernal = SVC(kernel='linear',C=2,gamma=0.002)
model_linear_kernal.fit(trainx, trainy)
model_linear_kernal.score(testx, testy)

In [ ]:
model_g = SVC(kernel='rbf',gamma=0.002,C=2)
model_g.fit(trainx, trainy)
model_g.score(testx, testy)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn import svm
clf= GridSearchCV(svm.SVC(gamma='auto'),{
'C':[1,2,10,20],
'kernel':['rbf','linear']
},cv=5, return_train_score=False)
clf.fit(trainx,trainy)
score=clf.predict(testx)
from sklearn.metrics import confusion_matrix
CM = confusion_matrix(testy,score)
CM

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn import svm
clf = GridSearchCV(svm.SVC(gamma='auto'), {
    'C': [1,2,10,20],
    'kernel': ['rbf','linear']
}, cv=5, return_train_score=False)
clf.fit(unibank.drop('Personal Loan',1),unibank["Personal Loan"])
clf.cv_results_

In [ ]:
df=pd.DataFrame(clf.cv_results_)
df

In [ ]:
df[['param_C','param_kernel','mean_test_score']]

**Naive Bayes**

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
NB = GaussianNB()

NB.fit(X , y)

NB_train_pred = NB.predict(X)
print(accuracy_score(y,NB_train_pred))

NB_test_pred = NB.predict(testx)
print(accuracy_score(testy,NB_test_pred))

**KNN**

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

knn_classifier = KNeighborsClassifier(algorithm='brute',weights='distance')
params = {'n_neighbors':[1,11,25],'metric':["euclidean",'cityblock']}
grid = GridSearchCV(knn_classifier,param_grid=params,scoring='accuracy',cv=10)

In [ ]:
grid.fit(trainx,trainy)
print(grid.best_score_)
print(grid.best_params_)

In [ ]:
best_knn = grid.best_estimator_
pred_train = best_knn.predict(trainx) 
pred_test = best_knn.predict(testx)
print("Accuracy on train is:",accuracy_score(trainy,pred_train))
print("Accuracy on test is:",accuracy_score(testy,pred_test))

**Decision Tree**

In [ ]:
from sklearn import tree

# Defining the model
# Fit / train the model
dtc = tree.DecisionTreeClassifier()
dtc.fit(trainx,trainy)

In [ ]:
# Get the prediction for both train and test
pred_train = dtc.predict(trainx)
pred_test = dtc.predict(testx)
# Measure the accuracy of the model for both train and test sets
print("Accuracy on train is:",accuracy_score(trainy,pred_train))
print("Accuracy on test is:",accuracy_score(testy,pred_test))

**Random Forest**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
print(rfc)

In [ ]:
rfc.fit(trainx,trainy)
## Predict
rfc_train_predictions = rfc.predict(trainx)
rfc_test_predictions = rfc.predict(testx)

### Train data accuracy
from sklearn.metrics import accuracy_score
print(accuracy_score(trainy,rfc_train_predictions))
      
### Test data accuracy
print(accuracy_score(testy,rfc_test_predictions))

In [ ]:
#Search Grid for Hyper paramenter tuning
from sklearn.model_selection import RandomizedSearchCV

## n_jobs = -1 uses all cores of processor
## max_features is the maximum number of attributes to select for each tree
rfc_grid = RandomForestClassifier(n_jobs=-1, max_features='sqrt', class_weight='balanced_subsample')
 
# Use a grid over parameters of interest
## n_estimators is the number of trees in the forest
## max_depth is how deep each tree can be
## min_sample_leaf is the minimum samples required in each leaf node for the root node to split
## "A node will only be split if in each of it's leaf nodes there should be min_sample_leaf"
 
param_grid = {"n_estimators" : [10, 25, 50, 75, 100],
           "max_depth" : [10, 12, 14, 16, 18, 20],
           "min_samples_leaf" : [5, 10, 15, 20]}
rfc_cv_grid = RandomizedSearchCV(estimator = rfc_grid, param_distributions = param_grid, cv = 3, n_iter=10)
rfc_cv_grid.fit(trainx, trainy)
rfc_cv_grid.best_estimator_

In [ ]:
## Predict
rfc2_train_predictions = rfc_cv_grid.predict(trainx)
rfc2_test_predictions = rfc_cv_grid.predict(testx)

print(accuracy_score(trainy,rfc2_train_predictions))
      
### Test data accuracy
print(accuracy_score(testy,rfc2_test_predictions))